# Fetch Data

Fetches all Anilist data needed for practicing data visualization and machine learning.

In [1]:
from anilist_data import fetch as anilist

anilist.get_user(247578)['data']['User']

Exception: Failed to request Anilist data.
Request body: {'query': 'query($id: Int!) {User(id: $id) {namecreatedAtstatistics {anime {countmeanScorestandardDeviationminutesWatchedepisodesWatched}manga {countmeanScorestandardDeviationchaptersReadvolumesRead}}avatar {large}}MediaListCollection(userId: $id, type: ANIME) {lists {namestatusentries {idstatusscore(format: POINT_10)progressprogressVolumescompletedAt {yearmonthday}media {id}}}}MediaListCollection(userId: $id, type: MANGA) {lists {namestatusentries {idstatusscore(format: POINT_10)progressprogressVolumescompletedAt {yearmonthday}media {id}}}}}', 'variables': {<built-in function id>: '247578'}}
<Response [400]>